Generate trip plan report

# Trip Report

## Configure OpenAI Settings

In [2]:
import os
import openai
from dotenv import load_dotenv
# Set up Azure OpenAI
load_dotenv()

openai.api_type = "azure"

AZURE_OPENAI_API_VERSION = os.getenv("AAG_AZURE_OPENAI_API_VERSION")
openai.api_version = AZURE_OPENAI_API_VERSION

AZURE_OPENAI_API_KEY = os.getenv("AAG_AZURE_OPENAI_API_KEY").strip()
assert AZURE_OPENAI_API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = AZURE_OPENAI_API_KEY

AZURE_OPENAI_ENDPOINT = os.getenv("AAG_AZURE_OPENAI_ENDPOINT", "").strip()
assert AZURE_OPENAI_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
openai.api_base = AZURE_OPENAI_ENDPOINT

# Deployment for Chat
# DEPLOYMENT_NAME_CHAT = os.getenv('DEPLOYMENT_NAME_CHAT')
DEPLOYMENT_NAME_CHAT = os.getenv('AAG_DEPLOYMENT_NAME_CHAT_16K')

# Deployment for embedding
DEPLOYMENT_NAME_EMBEDDING = os.getenv("AAG_DEPLOYMENT_NAME_EMBEDDING")
model: str = DEPLOYMENT_NAME_EMBEDDING

# Azure AI Search (Cognitive vector store)
vector_store_address: str = os.getenv("AAG_AZURE_SEARCH_SERVICE_ENDPOINT")
vector_store_password: str = os.getenv("AAG_AZURE_SEARCH_ADMIN_KEY")
# index_name: str = "langchain-vector-arxiv-physics"

# Deployment for embedding
BING_SUBSCRIPTION_KEY = os.getenv("BING_SUBSCRIPTION_KEY")

## Azure OpenAI ChatCompletion

### Read finds from previous processed data

In [4]:
file_path = './reports/report_prompt.txt'

try:
    with open(file_path, 'r') as file:
        content = file.read()
        print(content)
except FileNotFoundError:
    print("The file doesn't exist or the path is incorrect.")



I am going to plan a Ski SLC ski trip for a week. Here are the information on this trip:
1. I have booked airport parking: The parking information is as follows:
{
  "address": "14707 Guy R Brewer Boulevard, New York, United States",
  "rating": "4",
  "reviews": "539",
  "type": "Commercial Parking Lot or Garage",
  "operating_hours": {
    "Sunday": "24 hours",
    "Monday": "24 hours",
    "Tuesday": "24 hours",
    "Wednesday": "24 hours",
    "Thursday": "24 hours",
    "Friday": "24 hours",
    "Saturday": "24 hours"
  }
}
2. Booked flight: The flight information is as follows:
{
  "flight1": {
    "departure": "New York-JFK",
    "arrival": "Salt Lake City",
    "date": "Dec 6, 2023",
    "departureTime": "11:00 am",
    "arrivalTime": "02:15 pm",
    "duration": "5 hrs 15 mins",
    "stops": "Nonstop",
    "class": "Blue Basic",
    "flightNumber": "B6871",
    "operatedBy": "JetBlue"
  },
  "flight2": {
    "departure": "Salt Lake City",
    "arrival": "New York-JFK",
    "da

### Load JSON

In [3]:
flight_info = [{
  "ConfirmationCode": "IHHZZC",
  "Ticketed": "Yes",
  "Flights": [
    {
      "From": "New York-JFK",
      "To": "Salt Lake City",
      "Date": "Dec 6, 2023",
      "DepartureTime": "11:00 am",
      "ArrivalTime": "02:15 pm",
      "Duration": "5 hrs 15 mins",
      "FlightNumber": "B6871 A320",
      "OperatedBy": "JetBlue",
      "FlightType": "Nonstop, Blue Basic"
    },
    {
      "From": "Salt Lake City",
      "To": "New York-JFK",
      "Date": "Dec 12, 2023",
      "DepartureTime": "03:11 pm",
      "ArrivalTime": "09:38 pm",
      "Duration": "4 hrs 27 mins",
      "FlightNumber": "B6872 A320",
      "OperatedBy": "JetBlue",
      "FlightType": "Nonstop, Blue Basic"
    }
  ],
  "Note": "There has been a change to your flight. If this new flight does not work for you then use the 'Change' link on the JetBlue website."
}]


In [4]:
car_rental=[{
  "RentalCompany": "Fox",
  "ConfirmationNumber": "WF********",
  "ExpediaItinerary": "7267**********",
  "PickUp": {
    "Date": "Wed, Dec 6, 2023",
    "Time": "2:30pm"
  },
  "DropOff": {
    "Date": "Tue, Dec 12, 2023",
    "Time": "2:15pm"
  },
  "Note": "Please sign in to your Expedia account to see all booking details and manage your trip."
}]


In [5]:
airport_parking=[{
  "Location": "14707 Guy R Brewer Boulevard, New York, United States",
  "Rating": "4 (539 Ratings)",
  "ParkingType": "Outdoor Valet (Daily)",
  "Price": "From $7.50 per day",
  "CheckIn": "Wed, Dec 06, 09:00 AM",
  "CheckOut": "Tue, Dec 12, 11:30 PM",
  "OperatingHours": "Open 24 hours, 7 days a week",
  "Note": "This is a commercial parking lot or garage."
}]

In [6]:
hotel=[{
  "HotelName": "Fairfield Inn & Suites Salt Lake City South",
  "Address": "594 West 4500 South Salt Lake City Utah 84123 USA",
  "PhoneNumber": "+1-801-265-9600",
  "GuestName": "Mr. Yong",
  "StayDuration": {
    "CheckIn": "Wednesday, December 6, 2023 03:00 PM",
    "CheckOut": "Tuesday, December 12, 2023 11:00 AM"
  },
  "Rooms": {
    "Number": 1,
    "GuestsPerRoom": "2 Adults"
  },
  "GuaranteeMethod": "Credit Card Guarantee, Visa",
  "TotalForStay": "xxxx USD",
  "ConfirmationNumber": "84544955"
}]

### Run ChatCompletion with user prompt to generate trip reports

In [7]:
trip_name = 'Ski SLC'

# prompt_text = f"""
# Please generate an assessment report on {company_name} based on the following questions and answers in JSON:
# {data}
# and the assessment report should contain the following sections:
# 1. Executive Summary
# 2. Overview of Key Findings
# 3. Summary of Risks Identified
# 4. Overall ratings on the risk level you would give, between 0 and 9 (0 - no risk, 9 - highest risk)
# """


# prompt_text = f"""
# I am going to plan a {trip_name} ski trip for a week. Here are the information on this trip:
# 1. I have booked airport parking: {airport_parking}
# 2. Booked flight: {flight_info}
# 3. Booked hotel in the destination: {hotel}
# 4. Booked car rental in the destination: {car_rental}
# 5. We will ski in the following Ski resorts in UT: Solitude Mountain Resort, Brighton Resort, and Alta Snowbird. 

# With all above information, can you make a detailed day to day ski trip plan include following:
# 1. Entire trip planning starting from the every beginning, New York, like from parking, then fly out, then to SLC, then get Rental car, then go to hotel, 
# then Ski activities (filling in for the entire stay in this trip), then check out hotel, return rental car, take flight back.
# 2. Each day should include driving directions and address to one of those Ski resorts, where to park, and where to eat dinner.
# 3. For any location, address, contact info are needed.

# """
prompt_text = content
print(prompt_text)

response = openai.ChatCompletion.create(
    engine=DEPLOYMENT_NAME_CHAT,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt_text},
    ],
    max_tokens=15000,
)
report_content = response["choices"][0]["message"]["content"]

print(response["choices"][0]["message"]["content"])


I am going to plan a Ski SLC ski trip for a week. Here are the information on this trip:
1. I have booked airport parking: The parking information is as follows:
{
  "address": "14707 Guy R Brewer Boulevard, New York, United States",
  "rating": "4",
  "reviews": "539",
  "type": "Commercial Parking Lot or Garage",
  "operating_hours": {
    "Sunday": "24 hours",
    "Monday": "24 hours",
    "Tuesday": "24 hours",
    "Wednesday": "24 hours",
    "Thursday": "24 hours",
    "Friday": "24 hours",
    "Saturday": "24 hours"
  }
}
2. Booked flight: The flight information is as follows:
{
  "flight1": {
    "departure": "New York-JFK",
    "arrival": "Salt Lake City",
    "date": "Dec 6, 2023",
    "departureTime": "11:00 am",
    "arrivalTime": "02:15 pm",
    "duration": "5 hrs 15 mins",
    "stops": "Nonstop",
    "class": "Blue Basic",
    "flightNumber": "B6871",
    "operatedBy": "JetBlue"
  },
  "flight2": {
    "departure": "Salt Lake City",
    "arrival": "New York-JFK",
    "da

In [8]:
report_content

"Day 1: Arrival in Salt Lake City\n- Leave from the booked airport parking at 14707 Guy R Brewer Boulevard in New York.\n- Take the flight B6871 operated by JetBlue from New York-JFK to Salt Lake City.\n- Departure: 11:00 am from New York-JFK.\n- Arrival: 02:15 pm at Salt Lake City.\n- Duration: 5 hours 15 minutes, nonstop.\n- Upon arrival, proceed to the car rental location.\n- Pick up the rental car using the confirmation number #WF******** provided by the car rental company.\n- Drive to the Fairfield Inn & Suites Salt Lake City South hotel located at 594 West 4500 South, Salt Lake City, Utah 84123.\n- Check-in at the hotel at 03:00 PM.\n- After checking in, you have the evening free to explore the area or relax.\n\nDay 2: Skiing at Solitude Mountain Resort\n- Drive from the hotel to Solitude Mountain Resort located at 12000 Big Cottonwood Canyon Rd, Solitude, UT 84121.\n- Parking is available at the resort.\n- Enjoy a full day of skiing at Solitude Mountain Resort.\n- After skiing, 

In [9]:
word_file = './reports/trip_report_001.txt'

with open(word_file, 'w') as file:
    file.write(report_content)